# Preprocessing


In [25]:
from datasets import load_dataset
import pandas as pd
from datasets import concatenate_datasets, DatasetDict

### Dialogue Dataset

In [16]:
ds = load_dataset("squarelike/ko_medical_chat")
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations'],
        num_rows: 3038
    })
})

In [17]:
ds['train']['conversations'][0]

[{'from': 'client',
  'value': '안녕하세요, 의사님. 오늘 아침에 일어나서 의자에 앉아 있을 때 방 전체가 돌아가는 것 같은 느낌이 들었어요.'},
 {'from': 'doctor', 'value': '그런 증상이 있으셨군요. 이동할 때만 돌아가는 건가요?'},
 {'from': 'client', 'value': '네, 제가 움직이려고 하면 세상 전체가 돌아가는 것 같아요.'},
 {'from': 'doctor',
  'value': '그렇군요. 그리고 오전에 화장실에 가실 때 불안정하게 걸으셨다고 하셨는데, 그때 어떤 증상이 있었나요?'},
 {'from': 'client', 'value': '집중하기가 어려웠고, 메스꺼움을 느꼈어요. 구토를 시도했지만 나오지 않았어요.'},
 {'from': 'doctor', 'value': '그러셨군요. 그리고 복통도 있으셨다고 하셨는데, 어떤 종류의 복통이었나요?'},
 {'from': 'client',
  'value': '그냥 일반적인 복부 불쾌감이었어요. 하지만 방금 화장실에 갔다 온 후에는 돌아가는 증상이 완화되었어요.'},
 {'from': 'doctor', 'value': '감사합니다. 이 증상은 어제부터 있었나요?'},
 {'from': 'client', 'value': '아니요, 오늘 아침 처음으로 느껴졌어요.'},
 {'from': 'doctor',
  'value': '이해했습니다. 가장 가능성이 높은 원인은 양성돌발성 위치성 현훈(BPPV)입니다. 이 질환은 귀에서 발생하는 문제로 인해 발생하며, 몇 일 안에 스스로 호전됩니다. 베타히스틴 알약을 복용하면 증상이 완화될 것입니다. 전정 재활이나 적응 운동을 하면 증상의 재발을 예방할 수 있습니다. 이런 증상을 평가해 줄 이비인후과 전문의의 진료도 도'},
 {'from': 'doctor', 'value': '도움이 될 것입니다. 최선을 다하겠습니다. 어떻게 대처해야 할지 궁금하신가요?'},
 {'from': 'client', 

In [18]:
import random

def build_client_doctor_context(example):
    conv = example["conversations"]
    # client turn 인덱스 추출
    client_indices = [i for i, turn in enumerate(conv) if turn["from"] == "client"]
    # client turn이 2개 이상 있어야 함 (마지막 client 제외)
    if len(client_indices) < 2:
        return None

    # n: 마지막 client turn을 제외한 범위에서 랜덤 선택
    n = random.randint(1, len(client_indices) - 1)
    nth_client_idx = client_indices[n - 1]

    # context: 0 ~ nth_client_idx까지의 대화 누적
    context = []
    for turn in conv[:nth_client_idx + 1]:
        context.append({"from": turn["from"], "value": turn["value"]})

    # answer: 다음 doctor의 응답 찾기
    answer = None
    for turn in conv[nth_client_idx + 1:]:
        if turn["from"] == "doctor":
            answer = turn["value"]
            break

    if answer is None:
        return None

    return {
        "context": context,
        "answer": answer
    }


In [19]:
# 예시 적용 (ds['train'] 기준)
restructured = ds['train'].map(build_client_doctor_context)
# None 값 제거
restructured = restructured.filter(lambda x: x is not None)
restructured

Dataset({
    features: ['id', 'conversations', 'context', 'answer'],
    num_rows: 3020
})

In [32]:
restructured['context'][0]

[{'from': 'client',
  'value': '안녕하세요, 의사님. 오늘 아침에 일어나서 의자에 앉아 있을 때 방 전체가 돌아가는 것 같은 느낌이 들었어요.'},
 {'from': 'doctor', 'value': '그런 증상이 있으셨군요. 이동할 때만 돌아가는 건가요?'},
 {'from': 'client', 'value': '네, 제가 움직이려고 하면 세상 전체가 돌아가는 것 같아요.'}]

In [33]:
restructured.save_to_disk("/home/jungmin/workspace/project/rapa/data/dialogue_koMedicalChat_preprocessed") ## QA / Dialogue 데이터셋 모두 저장

Saving the dataset (0/1 shards):   0%|          | 0/3020 [00:00<?, ? examples/s]

### QA Dataset

In [ ]:
dentist = load_dataset("sean0042/KorMedMCQA", 'dentist') # 추가 전처리 필요
doctor = load_dataset("sean0042/KorMedMCQA", 'doctor') # 추가 전처리 필요
nurse = load_dataset("sean0042/KorMedMCQA", 'nurse') # 추가 전처리 필요
pharm = load_dataset("sean0042/KorMedMCQA", 'pharm') # 추가 전처리 필요

In [22]:
dentist

DatasetDict({
    train: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 297
    })
    dev: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 304
    })
    test: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 811
    })
    fewshot: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 5
    })
})

In [27]:
# dentist의 train, dev, test split을 합쳐서 하나의 train으로 통합
dentist_merged = DatasetDict({
    "train": concatenate_datasets([dentist["train"], dentist["dev"], dentist["test"]])
})
doctor_merged = DatasetDict({
    "train": concatenate_datasets([doctor["train"], doctor["dev"], doctor["test"]])
})
nurse_merged = DatasetDict({
    "train": concatenate_datasets([nurse["train"], nurse["dev"], nurse["test"]])
})
pharm_merged = DatasetDict({
    "train": concatenate_datasets([pharm["train"], pharm["dev"], pharm["test"]])
})

dentist_merged

DatasetDict({
    train: Dataset({
        features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot'],
        num_rows: 1412
    })
})

In [28]:
from datasets import concatenate_datasets, Dataset

# 각 데이터셋에 category 컬럼 추가
def add_category_column(dataset, category):
    return dataset["train"].map(lambda x: {**x, "category": category})

dentist_with_cat = add_category_column(dentist_merged, "dentist")
doctor_with_cat = add_category_column(doctor_merged, "doctor")
nurse_with_cat = add_category_column(nurse_merged, "nurse")
pharm_with_cat = add_category_column(pharm_merged, "pharm")

# 통합
all_merged_with_category = concatenate_datasets([
    dentist_with_cat,
    doctor_with_cat,
    nurse_with_cat,
    pharm_with_cat
])

all_merged_with_category

Map:   0%|          | 0/1412 [00:00<?, ? examples/s]

Map:   0%|          | 0/2489 [00:00<?, ? examples/s]

Map:   0%|          | 0/1751 [00:00<?, ? examples/s]

Map:   0%|          | 0/1817 [00:00<?, ? examples/s]

Dataset({
    features: ['subject', 'year', 'period', 'q_number', 'question', 'A', 'B', 'C', 'D', 'E', 'answer', 'cot', 'category'],
    num_rows: 7469
})

In [34]:
all_merged_with_category.save_to_disk("/home/jungmin/workspace/project/rapa/data/qa_korMedMCQA_preprocessed") ## QA / Dialogue 데이터셋 모두 저장

Saving the dataset (0/1 shards):   0%|          | 0/7469 [00:00<?, ? examples/s]